In [1]:
import pandas as pd 
import nltk
import re
from nltk.stem import PorterStemmer
import string
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from base_datos_airbnb import bd, extract_data_from_db, Airbnb 
from limpieza import limpiezadedatos
import plotly.graph_objects as go
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
df =extract_data_from_db()

In [3]:
df['city'].unique()

array(['San Francisco', 'Las vegas', 'Miami', 'Charlotte', 'Austin',
       'Chicago', 'Seattle'], dtype=object)

In [4]:
df = limpiezadedatos(df)

In [5]:
df['city'].unique()

array(['San Francisco', 'Las vegas', 'Miami', 'Charlotte', 'Austin',
       'Chicago', 'Seattle'], dtype=object)

In [6]:
df = df.dropna(subset=['all_reviews'])

## Limpiamos el texto

In [7]:
df_nlp = pd.DataFrame(df[['unique_id', 'all_reviews']])

In [8]:
def process_reviews(reviews_str):
    try:
        # Eliminar los corchetes y comillas al inicio y al final
        reviews__cleaned = reviews_str.strip("[]'\"")
        
        # Separar las reseñas por comas (o cualquier otro delimitador)
        reviews_list = reviews__cleaned.split("', '")
        
        # Limpiar espacios en blanco en cada reseña
        processed_reviews = [review.strip() for review in reviews_list]
        
        return processed_reviews
    except:
        return []

df_nlp['processed_reviews'] = df_nlp['all_reviews'].apply(process_reviews)


df_nlp.head()

,unique_id,all_reviews,processed_reviews
0,0019b8df-c2b4-4b22-8876-db5c2bb4ff9e,"['Nice place with a nice view of the city, esp...","[Nice place with a nice view of the city, espe..."
1,00432fb0-d9ac-4346-b9f7-6bd3ae18aeda,[],[]
2,0064b7c5-d957-42ca-ab61-e4ae8e36d013,"['Thanks again to Eva. Very responsive, met al...","[Thanks again to Eva. Very responsive, met all..."
3,009db955-2045-4a1a-8d2a-283c67670180,['We had an amazing stay! The place was so bea...,[We had an amazing stay! The place was so beau...
4,00b1a711-fad5-4688-b895-0c0e183c1c44,['Nice place to stay. Literally 15-20 mins fro...,[Nice place to stay. Literally 15-20 mins from...


In [9]:
df_nlp['processed_reviews'][0]

['Nice place with a nice view of the city, especially at night. Jason and Ryan are such a friendly host for everything.',
 'We had a comfortable time staying here, the location is great and they have a lovely view of the city from their balcony! Thank you for being amazing hosts!',
 'Jason and Ryan were such amazing hosts—helpful, friendly, and incredibly accommodating throughout my stay. They made me feel right at home, even though I was a stranger from a different country. Thank you both for your kindness and hospitality! Love from India.\', "Lovely house with killer views, 15 mins walk from Castro. It\'s the hosts\' actual residence and feels like a home in all the good ways. Quiet comfortable bedroom and spotless private bathroom with good shower. Jason and Ryan are friendly, helpful and considerate hosts and quick to offer recommendations on how to best enjoy SF. Loved my stay and would absolutely come back.", \'Definitely a 5 star stay! Both the place and the hosts were amazing. 

In [10]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[-—]+', '', text)
    stopwords = nltk.corpus.stopwords.words("english")
    text = ' '.join([word for word in text.split() if word not in stopwords])
    #Eliminar espacios en blanco
    text = re.sub(r'\s+', ' ', text).strip()
    return text


Noe he usado stemming porque daba palabra sin sentido 

In [11]:
df_nlp['cleaned_reviews'] = df_nlp['processed_reviews'].apply(lambda reviews: [clean_text(review) for review in reviews])

In [12]:
df_nlp['cleaned_reviews'][0]

['nice place nice view city especially night jason ryan friendly host everything',
 'comfortable time staying location great lovely view city balcony thank amazing hosts',
 'jason ryan amazing hostshelpful friendly incredibly accommodating throughout stay made feel right home even though stranger different country thank kindness hospitality love india lovely house killer views 15 mins walk castro hosts actual residence feels like home good ways quiet comfortable bedroom spotless private bathroom good shower jason ryan friendly helpful considerate hosts quick offer recommendations best enjoy sf loved stay would absolutely come back definitely 5 star stay place hosts amazing gorgeous view elevated entire experience spot always top list future trips sf',
 'best view spot sf right home jason’snmy boyfriend planned last minute trip sf yosemite found airbnb listing night jason quick respond morning shocked fast accept us everything ready us time arrivedncheck smooth easy place spotless clean

Limpiar las reviewds
Nota todo esto tienes que hacerlo por fila!


In [13]:
#Crear una fila por reseña para pdoer calcular el bag of words y td-idf individual y asi despues hacer el promedio  
df_expanded = df_nlp.explode('cleaned_reviews').reset_index(drop=True)

In [14]:
df_expanded

,unique_id,all_reviews,processed_reviews,cleaned_reviews
0,0019b8df-c2b4-4b22-8876-db5c2bb4ff9e,"['Nice place with a nice view of the city, esp...","[Nice place with a nice view of the city, espe...",nice place nice view city especially night jas...
1,0019b8df-c2b4-4b22-8876-db5c2bb4ff9e,"['Nice place with a nice view of the city, esp...","[Nice place with a nice view of the city, espe...",comfortable time staying location great lovely...
2,0019b8df-c2b4-4b22-8876-db5c2bb4ff9e,"['Nice place with a nice view of the city, esp...","[Nice place with a nice view of the city, espe...",jason ryan amazing hostshelpful friendly incre...
3,0019b8df-c2b4-4b22-8876-db5c2bb4ff9e,"['Nice place with a nice view of the city, esp...","[Nice place with a nice view of the city, espe...",best view spot sf right home jason’snmy boyfri...
4,0019b8df-c2b4-4b22-8876-db5c2bb4ff9e,"['Nice place with a nice view of the city, esp...","[Nice place with a nice view of the city, espe...",jason isnawesome great place great communicati...
...,...,...,...,...
105381,ffc9a2f7-37fd-4b22-9f3e-cbf5f054f284,['Enjoyed my stay here during the hectic CES w...,[Enjoyed my stay here during the hectic CES we...,perfect stay needed place rest 1 night area sa...
105382,ffc9a2f7-37fd-4b22-9f3e-cbf5f054f284,['Enjoyed my stay here during the hectic CES w...,[Enjoyed my stay here during the hectic CES we...,stayed multiple times pleasant experience time
105383,ffc9a2f7-37fd-4b22-9f3e-cbf5f054f284,['Enjoyed my stay here during the hectic CES w...,[Enjoyed my stay here during the hectic CES we...,great place stay
105384,ffc9a2f7-37fd-4b22-9f3e-cbf5f054f284,['Enjoyed my stay here during the hectic CES w...,[Enjoyed my stay here during the hectic CES we...,great location great host can’t wait stay


## Vectorizacion del texto y escalado

In [15]:

#obtener Bag-of-Words
vectorizer = CountVectorizer()
X_counts = vectorizer.fit_transform(df_expanded['cleaned_reviews'])

#escalar Bag-of-Words con TF-IDF
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

#convertir la matriz TF-IDF a array denso (si es necesario)
X_tfidf_dense = X_tfidf.toarray()

#agregar los vectores TF-IDF directamente como arrays NumPy en la columna del DataFrame
df_expanded['tfidf'] = list(X_tfidf_dense)  # Cada fila de 'tfidf' contiene un array NumPy

#agrupar por 'Unnamed: 0' y calcular la media de los vectores TF-IDF (usando NumPy directamente)
df_grouped = df_expanded.groupby('unique_id')['tfidf'].apply(lambda x: np.mean(np.vstack(x), axis=0))

#crear DataFrame con los resultados (sin convertir a lista)
df_result = pd.DataFrame({
    'unique_id': df_grouped.index,
    'mean_tfidf': df_grouped.values  # Esto se mantiene como arrays NumPy
})

# convertir la columna 'mean_tfidf' a una matriz
X= np.vstack(df_result['mean_tfidf'])
num_features = X.shape[1]
tfidf_column_names = [f'tfidf_{i}' for i in range(num_features)]

# Convertir la matriz en un dataframe para poder concatenarlo con el df_modelo
X_df = pd.DataFrame(X, columns=tfidf_column_names)



In [16]:
X_df

,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,tfidf_6,tfidf_7,tfidf_8,tfidf_9,...,tfidf_54882,tfidf_54883,tfidf_54884,tfidf_54885,tfidf_54886,tfidf_54887,tfidf_54888,tfidf_54889,tfidf_54890,tfidf_54891
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
print(df_expanded['cleaned_reviews'][0])

nice place nice view city especially night jason ryan friendly host everything


### Análisis de sentimiento textblob

In [18]:
from textblob import TextBlob

# Añadir columna de sentimiento (polarity de TextBlob, entre -1 y 1)
df_expanded['polarity'] = df_expanded['cleaned_reviews'].apply(lambda review: TextBlob(review).sentiment.polarity)
df_grouped_2 = df_expanded.groupby('unique_id')['polarity'].mean()
df_result_2 = pd.DataFrame({
    'unique_id': df_grouped_2.index,
    'mean_polarity_Textblob': df_grouped_2.values
})
df_result_2= df_result_2.drop(columns= 'unique_id')


In [19]:
df_result_2

,mean_polarity_Textblob
0,0.504742
1,0.000000
2,0.487487
3,0.481604
4,0.451618
...,...
1854,0.385249
1855,0.553768
1856,0.478064
1857,0.474697


### Análisis de sentimiento con VADER

In [20]:

# Descargar lexicon de VADER
nltk.download('vader_lexicon')

# Inicializar el analizador de sentimiento VADER
sia = SentimentIntensityAnalyzer()

# Función para obtener el puntaje compound de VADER (entre -1 y 1)
def vader_polarity(review):
    return sia.polarity_scores(review)['compound']

# Añadir columna de polaridad utilizando VADER
df_expanded['polarity'] = df_expanded['cleaned_reviews'].apply(vader_polarity)

# Agrupar por unique_id y calcular la media de polaridad
df_grouped_3 = df_expanded.groupby('unique_id')['polarity'].mean()

# Crear el DataFrame de resultado con la polaridad media
df_result_3 = pd.DataFrame({
    'unique_id': df_grouped_3.index,
    'mean_polarity_nltk': df_grouped_3.values
})

# Opcional: si deseas eliminar la columna 'unique_id'
df_result_3 = df_result_3.drop(columns='unique_id')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/silvanaruizmedina/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [21]:
df_result_3

,mean_polarity_nltk
0,0.938986
1,0.000000
2,0.843585
3,0.804562
4,0.773743
...,...
1854,0.810909
1855,0.831209
1856,0.871438
1857,0.776059


In [22]:
df_modelo = pd.concat([df, X_df, df_result_2, df_result_3], axis=1)

### Encoding de las variables categoricas

In [23]:
encoder = OneHotEncoder()


In [24]:
#encoding guest favorite
#guest_favourite_encoded = encoder.fit_transform(df_modelo[['guest_favorite']]).toarray()
#guest_favourite_columns = encoder.get_feature_names_out(['guest_favorite'])  # Obtener nombres de las columnas
#guest_favourite_df = pd.DataFrame(guest_favourite_encoded, columns=guest_favourite_columns)

#encoding type_host
type_host_encoded = encoder.fit_transform(df_modelo[['type_host']]).toarray()
type_host_columns = encoder.get_feature_names_out(['type_host'])
type_host_df = pd.DataFrame(type_host_encoded, columns=type_host_columns)

#encoding city
city_encoded = encoder.fit_transform(df_modelo[['city']]).toarray()
city_columns = encoder.get_feature_names_out(['city'])
city_df = pd.DataFrame(city_encoded, columns=city_columns)

#encoding type_bathroom
bathroom_encoded = encoder.fit_transform(df_modelo[['type_bathroom']]).toarray()
bathroom_columns = encoder.get_feature_names_out(['type_bathroom'])
bathroom_df = pd.DataFrame(bathroom_encoded, columns=bathroom_columns)

#### Tratado de NaNs


In [25]:
# Reemplazar los NaN con 0 en las columnas deseadas
df_modelo['number_bedroom'] = df_modelo['number_bedroom'].fillna(0)
df_modelo['number_beds'] = df_modelo['number_beds'].fillna(0)
df_modelo['number_guest'] = df_modelo['number_guest'].fillna(0)


### Escalado de las variables numéricas

In [26]:
scaler = MinMaxScaler()
df_modelo['number_reviews'] = scaler.fit_transform(df_modelo[['number_reviews']])
df_modelo['price'] = scaler.fit_transform(df_modelo[['price']])
df_modelo['hosting_time']= scaler.fit_transform(df_modelo[['hosting_time']])

In [27]:
df_modelo.head()

,unique_id,title,city,guest_favorite,rating,number_reviews,type_host,hosting_time,price,all_reviews,...,tfidf_54884,tfidf_54885,tfidf_54886,tfidf_54887,tfidf_54888,tfidf_54889,tfidf_54890,tfidf_54891,mean_polarity_Textblob,mean_polarity_nltk
0,0019b8df-c2b4-4b22-8876-db5c2bb4ff9e,"1 Private Bed, Renovated Bath, Castro & City V...",San Francisco,1.0,4.94,0.241319,Superhost,0.714286,0.128843,"['Nice place with a nice view of the city, esp...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.504742,0.938986
1,00432fb0-d9ac-4346-b9f7-6bd3ae18aeda,Pool and Arcade 4BR Vacation!,Las vegas,0.0,NaN,NaN,Superhost,0.035714,0.232796,[],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,0064b7c5-d957-42ca-ab61-e4ae8e36d013,Stunning 2Bed Waterview Downtown,Miami,1.0,4.91,0.046296,Superhost,0.714286,0.515373,"['Thanks again to Eva. Very responsive, met al...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.487487,0.843585
3,009db955-2045-4a1a-8d2a-283c67670180,Charming & Modern South Park Gem: Pool~Gym~Pkg,Charlotte,1.0,5.00,0.002315,normal_host,0.011905,0.098097,['We had an amazing stay! The place was so bea...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.481604,0.804562
4,00b1a711-fad5-4688-b895-0c0e183c1c44,Relaxing home in SouthEnd Area! Pets are welco...,Charlotte,0.0,4.69,0.134259,Superhost,0.500000,0.077599,['Nice place to stay. Literally 15-20 mins fro...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.451618,0.773743


In [28]:
df_modelo = df_modelo.drop(columns=['unique_id', 'city', 'type_host', 'title',  'all_reviews', 'url',  'type_bathroom'])

In [29]:
df_modelo.head()

,guest_favorite,rating,number_reviews,hosting_time,price,id_url,number_guest,number_bedroom,number_beds,number_bathroom,...,tfidf_54884,tfidf_54885,tfidf_54886,tfidf_54887,tfidf_54888,tfidf_54889,tfidf_54890,tfidf_54891,mean_polarity_Textblob,mean_polarity_nltk
0,1.0,4.94,0.241319,0.714286,0.128843,5547500,0.0,0.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.504742,0.938986
1,0.0,NaN,NaN,0.035714,0.232796,1269948790719996158,10.0,4.0,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,1.0,4.91,0.046296,0.714286,0.515373,765495225209376134,5.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.487487,0.843585
3,1.0,5.00,0.002315,0.011905,0.098097,1215360454497242015,6.0,2.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.481604,0.804562
4,0.0,4.69,0.134259,0.500000,0.077599,26846030,4.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.451618,0.773743


In [30]:
df_modelo=pd.concat([df_modelo, type_host_df, city_df, bathroom_df], axis=1)

In [31]:
#Drop filas que tienen valores nulos
df_modelo = df_modelo.dropna()

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1859 entries, 0 to 1859
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   unique_id        1859 non-null   object 
 1   title            1855 non-null   object 
 2   city             1859 non-null   object 
 3   guest_favorite   1859 non-null   int64  
 4   rating           1770 non-null   float64
 5   number_reviews   1718 non-null   float64
 6   type_host        1859 non-null   object 
 7   hosting_time     1859 non-null   int64  
 8   price            1669 non-null   float64
 9   all_reviews      1859 non-null   object 
 10  url              1859 non-null   object 
 11  id_url           1859 non-null   object 
 12  number_guest     1550 non-null   float64
 13  number_bedroom   1573 non-null   float64
 14  number_beds      1827 non-null   float64
 15  type_bathroom    1859 non-null   object 
 16  number_bathroom  1711 non-null   float64
dtypes: float64(7), int6

In [33]:
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432 entries, 2 to 1858
Columns: 54919 entries, guest_favorite to type_bathroom_nan
dtypes: float64(54918), object(1)
memory usage: 600.0+ MB


### Modelo

In [34]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import plotly.graph_objects as go


# Función para realizar GridSearch con los hiperparámetros apropiados según el modelo
def realizar_grid_search(modelo, X_train, y_train, nombre_modelo):
    # Definir el grid de hiperparámetros para cada modelo
    param_grid = {}

    if nombre_modelo == 'RandomForestRegressor':
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        }
    elif nombre_modelo == 'GradientBoostingRegressor':
        param_grid = {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        }
    elif nombre_modelo == 'AdaBoostRegressor':
        param_grid = {
            'n_estimators': [50, 100],
            'learning_rate': [0.01, 0.1, 1]
        }
    elif nombre_modelo == 'KNeighborsRegressor':
        param_grid = {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    elif nombre_modelo == 'SVR':
        param_grid = {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf', 'poly'],
            'gamma': ['scale', 'auto']
        }
    elif nombre_modelo == 'DecisionTreeRegressor':
        param_grid = {
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        }

    # Si no hay parámetros a ajustar, no se ejecuta GridSearch
    if not param_grid:
        return modelo

    # Usar GridSearchCV
    grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, scoring='r2', cv=10, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)

    # Obtener el mejor modelo ajustado
    mejor_modelo = grid_search.best_estimator_
    mejor_score = grid_search.best_score_

    print(f"Mejor modelo encontrado para {nombre_modelo}: {mejor_modelo}")
    print(f"Mejor score (R2) en GridSearchCV: {mejor_score}")

    return mejor_modelo


# Función para graficar predicciones vs valores reales
def graficar_predicciones_vs_reales(y_test, y_pred, nombre_modelo):
    # Crear la figura
    fig = go.Figure()

    # Añadir la traza para los valores reales
    fig.add_trace(go.Scatter(
        x=y_test,
        y=y_pred,
        mode='markers',
        name='Predicciones',
        marker=dict(color='blue', size=10),
        hovertext=[f'Real: {real}<br>Predicción: {pred}' for real, pred in zip(y_test, y_pred)]
    ))

    # Añadir la línea de referencia (y=x) para facilitar la comparación
    fig.add_trace(go.Scatter(
        x=y_test,
        y=y_test,
        mode='lines',
        name='Valor Real',
        line=dict(color='red', dash='dash')
    ))

    # Configurar el layout del gráfico
    fig.update_layout(
        title=f'Predicciones vs Valores Reales - {nombre_modelo}',
        xaxis_title='Valores Reales',
        yaxis_title='Predicciones',
        width=700,
        height=500,
        showlegend=True,
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True)
    )

    # Mostrar el gráfico
    fig.show()


def evaluate_models(X, y, df_original, aplicar_gridsearch=True):
    # Suponemos que el df_original contiene la columna 'ID'
    
    ids = df_original['id_url']

    # Eliminar la columna 'ID' de X antes de entrenar los modelos
    X_sin_id = X.drop(columns=['id_url'])

    # Lista de modelos de regresión
    modelos = [
        LinearRegression(),
        KNeighborsRegressor(),
        DecisionTreeRegressor(),
        RandomForestRegressor(),
        SVR(),
        AdaBoostRegressor(),
        GradientBoostingRegressor()
    ]

    nombres = [
        'LinearRegression',
        'KNeighborsRegressor',
        'DecisionTreeRegressor',
        'RandomForestRegressor',
        'SVR',
        'AdaBoostRegressor',
        'GradientBoostingRegressor'
    ]

    # DataFrame para acumular los resultados
    resultados = pd.DataFrame(columns=['Model', 'R2', 'RMSE', 'MAE'])
    
    # Definir variables para guardar el mejor modelo
    mejor_r2 = -np.inf  # R2 más alto es mejor
    mejor_y_test = None
    mejor_y_pred = None
    mejor_ids_test = None  # Almacenar los IDs correspondientes
    mejor_modelo = None
    mejor_nombre = None
    mejor_X_train = None
    mejor_y_train = None

    # Entrenar y evaluar cada modelo
    for modelo, nombre in zip(modelos, nombres):
        # División del dataset (manteniendo el índice para rastrear los IDs)
        X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
            X_sin_id, y, ids, test_size=0.2, random_state=42)

        # Entrenar el modelo
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        # Métricas para evaluar el modelo
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)

        # Mejor modelo según el R2
        if r2 > mejor_r2:
            mejor_r2 = r2
            mejor_y_test = y_test
            mejor_y_pred = y_pred
            mejor_ids_test = ids_test  # Guardar los IDs correspondientes a este split
            mejor_modelo = modelo
            mejor_nombre = nombre
            mejor_X_train = X_train
            mejor_y_train = y_train

        # Crear un DataFrame con los resultados del modelo actual
        modelo_df = pd.DataFrame({
            'Model': [nombre],
            'R2': [r2],
            'RMSE': [rmse],
            'MAE': [mae]
        })

        # Mostrar las métricas
        print(f'Modelo: {nombre}\nR2: {r2}, RMSE: {rmse}, MAE: {mae}\n')

        resultados = pd.concat([resultados, modelo_df], ignore_index=True)

    # Ahora que hemos encontrado el mejor modelo, aplicar GridSearch solo a ese modelo
    if aplicar_gridsearch:
        print(f"\nAplicando GridSearchCV al mejor modelo: {mejor_nombre}")
        mejor_modelo = realizar_grid_search(mejor_modelo, mejor_X_train, mejor_y_train, mejor_nombre)

    # Imprimir los valores de y_test y y_pred del mejor modelo
    resultados_comparados = np.column_stack((mejor_y_test, mejor_y_pred))
    print("\nValores reales vs predicciones del mejor modelo:")
    print(resultados_comparados)

    # Crear un DataFrame para visualizar ID, y_test y y_pred
    df_resultado = pd.DataFrame({
        'ID': mejor_ids_test,
        'Valor Real': mejor_y_test,
        'Predicción': mejor_y_pred
    })

    # Mostrar el DataFrame con los resultados
    print(f"\nResultados del mejor modelo {mejor_nombre} con IDs correspondientes:")
    print(df_resultado)

    # Graficar predicciones vs valores reales
    graficar_predicciones_vs_reales(mejor_y_test, mejor_y_pred, mejor_nombre)

    return df_resultado




In [35]:
print(df_modelo.isnull().sum())

guest_favorite           0
rating                   0
number_reviews           0
hosting_time             0
price                    0
                        ..
city_nan                 0
type_bathroom_private    0
type_bathroom_shared     0
type_bathroom_unknown    0
type_bathroom_nan        0
Length: 54919, dtype: int64


In [36]:
#Llamamos a la funcion 
X=df_modelo.drop(columns= ['rating'])
y = df_modelo['rating']
resultados= evaluate_models(X=X, y=y, df_original=df_modelo)

Modelo: LinearRegression
R2: -6.0520013592139704e+16, RMSE: 27226267.040824424, MAE: 19684686.720504988



/var/folders/7h/qmh6bk5x34g5j30ygn0wjywr0000gn/T/ipykernel_4071/2853864172.py:193: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  resultados = pd.concat([resultados, modelo_df], ignore_index=True)


Modelo: KNeighborsRegressor
R2: 0.49677196790035416, RMSE: 0.07850928172562255, MAE: 0.05644599303135886

Modelo: DecisionTreeRegressor
R2: 0.3421279406520118, RMSE: 0.08976547065132762, MAE: 0.06111498257839727

Modelo: RandomForestRegressor
R2: 0.6838293651674703, RMSE: 0.06222993638578148, MAE: 0.042752961672473805

Modelo: SVR
R2: 0.5892533950354794, RMSE: 0.0709292851064881, MAE: 0.05616313226699309

Modelo: AdaBoostRegressor
R2: 0.6029533957248285, RMSE: 0.06973636955571025, MAE: 0.05297737013956958

Modelo: GradientBoostingRegressor
R2: 0.6655529757368304, RMSE: 0.06400328419881876, MAE: 0.04508664385366439


Aplicando GridSearchCV al mejor modelo: RandomForestRegressor
Fitting 10 folds for each of 24 candidates, totalling 240 fits


/Users/silvanaruizmedina/Desktop/Bootcamp/airbnb/venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Mejor modelo encontrado para RandomForestRegressor: RandomForestRegressor(max_depth=20, min_samples_split=5, n_estimators=200)
Mejor score (R2) en GridSearchCV: 0.580687781796286

Valores reales vs predicciones del mejor modelo:
[[4.97   4.951 ]
 [4.96   4.9481]
 [4.94   4.9561]
 [4.93   4.9302]
 [4.76   4.7662]
 [4.94   4.9255]
 [4.93   4.9294]
 [4.94   4.9384]
 [4.76   4.7754]
 [4.99   4.9659]
 [4.88   4.893 ]
 [4.73   4.7424]
 [4.92   4.9625]
 [4.96   4.9578]
 [4.76   4.749 ]
 [4.84   4.7821]
 [4.62   4.7132]
 [4.91   4.9193]
 [4.89   4.9106]
 [4.99   4.9497]
 [5.     4.956 ]
 [5.     4.948 ]
 [5.     4.9643]
 [4.79   4.7704]
 [4.94   4.9482]
 [4.93   4.9196]
 [4.89   4.7858]
 [4.94   4.9552]
 [4.82   4.7817]
 [4.85   4.9316]
 [4.93   4.9309]
 [5.     4.9624]
 [4.93   4.9552]
 [4.9    4.766 ]
 [4.97   4.9402]
 [4.94   4.9209]
 [5.     4.956 ]
 [4.98   4.9614]
 [5.     4.9608]
 [4.86   4.775 ]
 [4.94   4.9254]
 [4.96   4.9315]
 [5.     4.9539]
 [4.92   4.9502]
 [4.84   4.932 ]
 [4.73

In [37]:
resultados.rename(columns={'ID': 'id_url'}, inplace=True)
#Guardamos los resultados para añadirlos a streamlit
resultados.to_csv('resultados_nlp.csv')

In [38]:
resultados

,id_url,Valor Real,Predicción
159,41819916,4.97,4.9510
1554,11589978,4.96,4.9481
595,50217447,4.94,4.9561
452,982771884533129815,4.93,4.9302
1381,8637174,4.76,4.7662
...,...,...,...
1844,41619718,4.98,4.9195
1206,54192684,4.95,4.9140
1166,1225671132635344385,5.00,4.9900
1036,1096306523761844493,5.00,4.9816
